In [1]:
from common_imports import *

# Data Loading

In [2]:
num_images = 292
image_container_holiday_damage = []
image_container_holiday_damage = [cv2.imread('pre_holiday_damage_images_2025/' + str(i).zfill(4) + '.tiff') for i in range(num_images)]

In [3]:
num_images = 1810
image_container_recent_damage = []
image_container_recent_damage = [cv2.imread('post_holiday_damage_images_2025/' + str(i).zfill(4) + '.tiff') for i in range(2,num_images)] # first two images have readout errors


In [ ]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Add the image
fig.add_trace(go.Image(z=image_container_holiday_damage[0]))

# Customize layout
fig.update_layout(
    title="Starting Beam Mode from holiday Damage",
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
)

# Show the plot
fig.show()

In [ ]:
# Create a Plotly figure
fig = go.Figure()

# Add the image
fig.add_trace(go.Image(z=image_container_recent_damage[-1]))

# Customize layout
fig.update_layout(
    title="Ending Beam Mode from Recent Damage",
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
)

# Show the plot
fig.show()

In [ ]:

# Create a Plotly figure
fig = go.Figure()

# Add the image
fig.add_trace(go.Image(z=image_container_holiday_damage[0]))

# Customize layout
fig.update_layout(
    title="Starting Beam Mode from Recent Damage",
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
)

# Show the plot
fig.show()


In [ ]:
# Create a Plotly figure
fig = go.Figure()

# Add the image
fig.add_trace(go.Image(z=image_container_holiday_damage[-1]))

# Customize layout
fig.update_layout(
    title="Ending Beam Mode from Recent Damage",
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
)

# Show the plot
fig.show()

# Track Threshold Differences Across Frames (2025-01-15)
Note that it is not obvious which parameters to use for the threshold differencing. For sake of an example, max_difference = 20.0, max_area = 10.0, low_intensity = 30 was used.  This depends on the metadata associated with run typically since exposure time and gain affect the recorded signal and noise floor in particular.  We can make an ansatz for the max_difference since damage should drastically change the pixel value.  This can also be done for max_area, but it would benefit more from exploration in the future since it's not clear what percentage should count as damaged.  

## Most Recent Damage

In [ ]:
from frame_diff_threshold_analysis import threshold_differencing
max_area = 10
max_difference = 20
low_intensity = 20
changed_pixels_container_recent_damage = []
damage_flagged_container_recent_damage = []
debug_count = 0
for i in range(len(image_container_recent_damage)-1):
    debug_count += 1
    print(f"Processing image {debug_count}")
    damaged, count = threshold_differencing(image_container_recent_damage[i], image_container_recent_damage[i+1], max_difference, max_area, low_intensity)
    changed_pixels_container_recent_damage.append(count)
    damage_flagged_container_recent_damage.append(damaged)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(num_images), y=changed_pixels_container_recent_damage, mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='Beam Damage Analysis (Recent)', xaxis_title='Frame Number', yaxis_title='Pixel Count Above max_difference')

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(num_images), y=damage_flagged_container_recent_damage, mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='Beam Damage Analysis (Recent)', xaxis_title='Frame Number', yaxis_title='Damage Flagged')

fig.show()

## Pre-Holiday Damage 

In [ ]:
max_area = 10
max_difference = 20
low_intensity = 20
changed_pixels_container_holiday_damage = []
damage_flagged_container_holiday_damage = []
debug_count = 0
for i in range(len(image_container_holiday_damage)-1):
    debug_count += 1
    print(f"Processing image {debug_count}")
    damaged, count = threshold_differencing(image_container_holiday_damage[i], image_container_holiday_damage[i+1], max_difference, max_area, low_intensity)
    changed_pixels_container_holiday_damage.append(count)
    damage_flagged_container_holiday_damage.append(damaged)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(num_images), y=changed_pixels_container_holiday_damage, mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='Beam Damage Analysis (Pre-Holiday)', xaxis_title='Frame Number', yaxis_title='Pixel Count Above max_difference')

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(num_images), y=damage_flagged_container_holiday_damage, mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='Beam Damage Analysis (Pre-Holiday)', xaxis_title='Frame Number', yaxis_title='Damage Flagged')

fig.show()

## Review
Ultimately, when this algorithm was tuned using the cpp program in the front high rep-rate ABL room, parameters were calibrated for the camera positioning and acquisition parameters.  The metadata was not captured in the damaged images present in this notebook.  The graphs showing the pixel count above the max_difference show that the more recent damage occurrence could potentially have been stopped (doesn't look quite as noisy), but the second damage occurrence (pre-holiday) pixel count time-series just looks like a noisy graph.

## Next Steps
It would be nice to look through 10-20 parameter values of the max_area and/or max_difference.  An entire quasi-ablation study on all parameters could be done as well, but max_area would seem to be the most important parameter to focus on.

We can also look at the difference between images delayed by a few frames, e.g. 50 back

A sort of contour analysis following basically the floodFind algorithm could be done as well, or the code from the original cpp program could be revisited to see if parameter tuning could be skipped and the contour finding is already implemented.

Beyond that, using a python library for image processing beyond opencv could be useful -- something which would have great performance although maybe only operating at lower rate.  Ailia could be a good choice, and there should be a free tier or python version of it.  Starting with Fourier analysis perhaps and looking at things like contour analysis to get better statistics for descerning the damage.



# Delayed Frame Difference Threshold Analysis

In [ ]:
max_area = 10
max_difference = 20
low_intensity = 20
changed_pixels_container_recent_damage = []
damage_flagged_container_recent_damage = []
debug_count = 0
for i in range(50,len(image_container_recent_damage)-1):
    debug_count += 1
    print(f"Processing image {debug_count}")
    damaged, count = threshold_differencing(image_container_recent_damage[i-50], image_container_recent_damage[i], max_difference, max_area, low_intensity)
    changed_pixels_container_recent_damage.append(count)
    damage_flagged_container_recent_damage.append(damaged)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(num_images), y=changed_pixels_container_recent_damage, mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='Beam Damage Analysis (Recent)', xaxis_title='Frame Number', yaxis_title='Pixel Count Above max_difference')

fig.show()

# FFT Analysis (2025-01-16)
Using numpy's built in fft function we can analyze the images for some kind of indication of the formation of rings.  A window is applied first to reduce edge effects of the fft, namely the Hanning window
$$
    w(n) = 0.5 \left( 1 - cos \left( \frac{2 n \pi}{M-1} \right) \right)
$$             
Where $M$ is the length of one side of the pixel array and n is the index. In two dimensions we can build the window as
$$
    w(n_x,n_y) = \sqrt{w(n_x)w(n_y)}
$$
 The fft is then computed and shifted to the center of the image, and the magnitude spectrum is computed.  This magnitude spectrum is then enhanced using a power law transformation to enhance faint features.  The spectrum is then normalized to [0,1].

## Testing on a single image


In [4]:
from fft_analysis import *

In [50]:
fig, magnitude_spectrum = analyze_diffraction_rings(image_container_recent_damage[0])

Magnitude range: 0.0011 to 7178.6448
Enhanced range: 0.0000 to 1.0000


In [ ]:
fig.show()
fig.write_html("single_image_normalized_fft.html")


## FTT Analysis Applied to Pre-Holiday Damage

In [ ]:
analyze_multiple_images(image_container_holiday_damage, 10, 12)

In [ ]:
analyze_multiple_images(image_container_holiday_damage, 150, 152)

In [ ]:
analyze_multiple_images(image_container_holiday_damage, 245, 247)

## FTT Analysis Applied to Recent Damage

In [ ]:
analyze_multiple_images(image_container_recent_damage, 10, 12)

In [ ]:
analyze_multiple_images(image_container_recent_damage, 1000, 1002)

In [ ]:
analyze_multiple_images(image_container_recent_damage, 1800, 1802)

## Spectrum Broadness Analysis
Seeing as the damage from before the holidays yielded a significant broadening towards the end of the collected sample, this should be able to be discerned from a comparison of broadening. This is the main deliverable for any decision algorithm, so it's important to compare options.  Three different basic methods were chosen to analyze the broadening of the spectrum and compared:
* second moment: ought to be pretty quick

* high frequency power ratio: tunable threshold which sums pixels outside a circle around the center (default chosen at 1/4 of the maximum distance, but not sure exactly what will work best)

* radial std: possible the most robust, but potentially also slower



### Check Metric Functionality and Performance 
using just a single image, the functions can be compared for runtime and checked for correctness.

In [ ]:
from spectrum_broadness_analysis import *
fig, magnitude_spectrum = analyze_diffraction_rings(image_container_holiday_damage[5])
metrics = analyze_spectrum_broadness(magnitude_spectrum)

In [ ]:
# Example dictionary (assuming this is similar to your data)
data_dict = metrics.copy()
# Convert to DataFrame
df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['broadness', 'ms'])

# Method 1: Simple scatter plot with pandas
df.plot(kind='scatter', x='broadness', y='ms')
plt.title('Normalized Spectrum Broadness and Time Analysis Visualization')
for i, row in df.iterrows():
    plt.annotate(i, (row['broadness'], row['ms']))

# Method 2: Using seaborn (often prettier)
sns.scatterplot(data=df, x='broadness', y='ms')
for i, row in df.iterrows():
    plt.annotate(i, (row['broadness'], row['ms']))

plt.show()

### Compare Across Pre-Holiday Damage Set

In [ ]:
fig, magnitude_spectrum_post_damage = analyze_diffraction_rings(image_container_holiday_damage[260])
metrics_pre_damage, metrics_post_damage = compare_images(magnitude_spectrum,magnitude_spectrum_post_damage)

### Compare Across Recent Damage Data

In [ ]:
_, magnitude_spectrum_start = analyze_diffraction_rings(image_container_recent_damage[5])
_, magnitude_spectrum_end = analyze_diffraction_rings(image_container_recent_damage[1800])
metrics_pre_damage_recent, metrics_post_damage_recent = compare_images(magnitude_spectrum_start, magnitude_spectrum_end)

### Time-Series Analysis for Discrimination


#### Pre-Holiday Damage Times-Series Analysis

In [ ]:
metrics_container_holiday_damage = []
for img in image_container_holiday_damage:
    _, spectrum = analyze_diffraction_rings(img)
    metrics_container_holiday_damage.append(analyze_spectrum_broadness(spectrum))


In [ ]:
radial_stds_holiday_damage = [metric['radial_std'][0] for metric in metrics_container_holiday_damage]

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(num_images), y=radial_stds_holiday_damage, mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='Beam Damage Analysis (Pre-Holiday)', xaxis_title='Frame Number', yaxis_title='radial_std of fft spectrum')

fig.show()

#### Recent Damage Time-Series Analysis

In [ ]:
metrics_container_recent_damage = []
for img in image_container_recent_damage:
    _, spectrum = analyze_diffraction_rings(img)
    metrics_container_recent_damage.append(analyze_spectrum_broadness(spectrum))


In [ ]:
radial_stds_recent_damage = [metric['radial_std'][0] for metric in metrics_container_recent_damage]

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(num_images), y=radial_stds_recent_damage, mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='Beam Damage Analysis (Recent)', xaxis_title='Frame Number', yaxis_title='radial_std of fft spectrum')

fig.show()

#### Average Runtime of Radial Std

In [ ]:
print(f"radial_std has a runtime of {np.average([metric['radial_std'][1] for metric in metrics_container_recent_damage]):.4f} ms on average")

## Review
Analysis shows significant broadening of the peak in the frequency spectrum, representative of the diffraction rings present. Further choice of metrics to compare broadening showed that the radial standard deviation is the most robust, despite being slower. 

Clearly the time-series graph of the radial standard deviation of the pre-holiday damage sample delineates the broadening of the spectrum and the formation of the diffraction rings. This could be used as a basis for a decision algorithm to determine whether to shut the diode off.

Because the more recent damage sample doesn't have any strong diffraction features across the set, the time-series graph of the fft spectrum's broadening doesn't -- and should not -- produce a signal which a decision algorithm could use to shut the diode off.

## Next Steps
The decision algorithm still has to be decided on.  Either a derivative function that averages over a moving window, which would be efficient enough, or simply a comparison to a discrimiator could be useful directions to go in.


# Indicator Implementation (2025-01-21)
The basic outline of the indicator is that it average the derivative (first or second order) over a rolling window of the buffer, going a certain number of frames back.  It will simply look for a non-negative value in the derivative.  Bryan specified that the function should return a boolean value, and that the parameters of the function should just be the image buffer itself, or the image container.  The best solution would just to be to assume it's the buffer, not the entire container. This solution is somewhere in between what was discussed briefly on last Friday morning.   

## Derivative Indicator Applied to Pre-Holiday Damage Data

In [ ]:
metrics_derivative_analysis_holiday_damage = detect_damage_from_timeseries(radial_stds_holiday_damage)

# Print results
print(f"Damage detected: {metrics_derivative_analysis_holiday_damage['damage_detected']}")
print("\nFirst Derivative Metrics:")
for key, value in metrics_derivative_analysis_holiday_damage['first_deriv_metrics'].items():
    print(f"{key}: {value:.6f}")
print("\nSecond Derivative Metrics:")
for key, value in metrics_derivative_analysis_holiday_damage['second_deriv_metrics'].items():
    print(f"{key}: {value:.6f}")

fig = plot_detection_results(metrics_derivative_analysis_holiday_damage)
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(len(metrics_derivative_analysis_holiday_damage['first_deriv_detection'])), y=metrics_derivative_analysis_holiday_damage['first_deriv_detection'], mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='First Derivative Indicator Time-Series (Pre-Holiday)', xaxis_title='Frame Number', yaxis_title='dammage detected')

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(len(metrics_derivative_analysis_holiday_damage['second_deriv_detection'])), y=metrics_derivative_analysis_holiday_damage['second_deriv_detection'], mode='lines+markers', marker=dict(size=5)))

fig.update_layout(title='Second Derivative Indicator Time-Series (Pre-Holiday)', xaxis_title='Frame Number', yaxis_title='dammage detected')

fig.show()